In [ ]:
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import inspect

In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, MetaData, Table

In [ ]:
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [ ]:
Base = automap_base()

Base.prepare(autoload_with= engine)

In [ ]:
Base.classes.keys()

In [ ]:
Station = Base.classes.station

Measurement = Base.classes.measurement

In [ ]:
session =  Session(engine)

In [ ]:
# Find the most recent date in the data set.
recent_date =(session.query(Measurement.date)
             .order_by(Measurement.date.desc())
                .first())
recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
one_year= dt.date(2017, 8, 23)-dt.timedelta(days=365)


# Calculate the date one year from the last date in data set.

prev_last_date = dt.date(one_year.year, one_year.month, one_year.day)


# Perform a query to retrieve the data and precipitation scores

results= session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_last_date).order_by(Measurement.date).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names

data_df =pd.DataFrame(results, columns=['date', 'precipitation'])

# Sort the dataframe by date

data_df.set_index(data_df['date'], inplace =True)

#drop N/A cells
clean_data_df = data_df.dropna()

#data_df.head()

# Use Pandas Plotting with Matplotlib to plot the data

clean_data_df.plot('date', 'precipitation')


plt.xlabel("Date",  size = 13)
plt.ylabel("Inches", size = 13)
plt.title("Hawaii Precipitation Aug 2016 - Aug 2017", size = 13)
plt.legend(["precipitation"],loc='upper center')
plt.xticks(rotation=90, size = 8)
plt.yticks(size = 8)
plt.show()


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
clean_data_df.describe()

In [ ]:
# Design a query to calculate the total number of stations in the dataset
session.query(Station.id).count()

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.

session.query(Measurement.station,func.count(Measurement.station)).group_by(Measurement.station)\
.order_by(func.count(Measurement.station).desc()).all()

In [ ]:
#Answer the following question: which station id has the greatest number of observations?
#USC00519281 station has the greatest number of observations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

session.query(func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs))\
    .filter(Measurement.station=='USC00519281').all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

results = session.query(Measurement.tobs).filter(Measurement.station=='USC00519281')\
    .filter(Measurement.date>='2016-08-23').all()

#Plot results
high_station_temp=pd.DataFrame(results,columns=['tobs'])
high_station_temp.plot.hist(bins=12)
plt.tight_layout()
plt.title("12 month Temperature Histogram", size=13)
plt.xlabel("Temperature")
plt.ylabel("Frequency")

plt.show()